## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sqlite3

## Read Data

In [2]:
con = sqlite3.connect("../data/nfts.sqlite")
cur = con.cursor()

## List All Available Tables

In [3]:
query = cur.execute("""SELECT * 
FROM sqlite_master 
WHERE type='table'""")
cols = [column[0] for column in query.description]
results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

,type,name,tbl_name,rootpage,sql
0,table,nfts,nfts,2,CREATE TABLE nfts\n (\n address TEXT...
1,table,checkpoint,checkpoint,5,CREATE TABLE checkpoint\n (\n event_...
2,table,mints,mints,6,CREATE TABLE mints\n (\n event_id TE...
3,table,transfers,transfers,8,CREATE TABLE transfers\n (\n event_i...
4,table,transfer_values_quartile_10_distribution_per_a...,transfer_values_quartile_10_distribution_per_a...,10,CREATE TABLE transfer_values_quartile_10_distr...
5,table,current_owners,current_owners,11,CREATE TABLE current_owners(\n nft_address TE...
6,table,current_market_values,current_market_values,12,CREATE TABLE current_market_values(\n nft_add...
7,table,market_values_distribution,market_values_distribution,13,CREATE TABLE market_values_distribution(\n ad...
8,table,transfer_statistics_by_address,transfer_statistics_by_address,14,CREATE TABLE transfer_statistics_by_address(\n...
9,table,transfer_values_quantile_10_distribution_per_a...,transfer_values_quantile_10_distribution_per_a...,15,CREATE TABLE transfer_values_quantile_10_distr...


## Union All Transactions

In [ ]:
# mints: 6667282
# transfers: 4514729
# combined: 11182011
query = cur.execute("""
SELECT *, 'mint' AS activity_type
FROM mints
""")
cols = [column[0] for column in query.description]
mints_df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
mints_df.head()

In [ ]:
query = cur.execute("""
SELECT *, 'transfer' as activity_type
FROM transfers
""")
cols = [column[0] for column in query.description]
transfers_df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
transfers_df.head()

In [ ]:
transactions_df = pd.concat([mints_df, transfers_df])
transactions_df.shape

In [ ]:
transactions_df = transactions_df.sort_values(by=['timestamp'], ascending=[True])
transactions_df.head()

In [ ]:
transactions_df['token_transaction_order'] = transactions_df.groupby(['nft_address','token_id']).cumcount()+1

In [ ]:
transactions_df

In [ ]:
transactions_df[transactions_df["transaction_value"] == 0.0]

# Questions

In [4]:
def get_results(cur: sqlite3.Cursor = cur, statement: str = '') -> pd.DataFrame:
    '''
    Returns results from sqlite query, in the form of a pandas dataframe.
    '''
    query = cur.execute(statement)
    cols = [col[0] for col in query.description]
    df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
    return df

In [19]:
# who has made the most money (takes 3min 16sec)
statement = '''
with
    mints_and_transfers as (
        select
            *
            , 'mint' as activity
        from mints
        union all
        select
            *
            , 'transfer' as activity
        from transfers
    )
    , nfts_with_order as (
        select
            transaction_hash
            , nft_address
            , token_id
            , activity
            , from_address
            , to_address
            , transaction_value
            , timestamp
            , row_number() over (
                partition by nft_address, token_id
                order by timestamp asc
            ) as row_num
        from mints_and_transfers
    )
    , starts_join_ends as (
        select
            nwo1.transaction_hash
            , nwo1.nft_address
            , nwo1.token_id
            , nwo1.activity as start_activity
            , nwo1.to_address as start_address
            , nwo1.transaction_value as start_value
            , nwo1.timestamp as start_timestamp
            , nwo1.row_num as start_row
            , ifnull(nwo2.activity, 'hold') as end_activity
            , ifnull(nwo2.to_address, nwo1.to_address) as end_address
            , ifnull(nwo2.transaction_value, cmv.market_value) as end_value
            , ifnull(nwo2.timestamp, 1632586540) as end_timestamp
            , ifnull(nwo2.row_num, nwo1.row_num) as end_row
        from nfts_with_order as nwo1
        left join nfts_with_order as nwo2 on 
            nwo1.nft_address = nwo2.nft_address
            and nwo1.token_id = nwo2.token_id
            and nwo1.row_num + 1 = nwo2.row_num
        left join current_market_values as cmv on
            nwo1.nft_address = cmv.nft_address
            and nwo1.token_id = cmv.token_id
    )
    , pairs_with_deltas as (
        select
            *
            , end_value - start_value as delta_value
            , julianday(end_timestamp, 'unixepoch') - julianday(start_timestamp, 'unixepoch') as delta_days
            , cast(start_row as string) || '->' || cast(end_row as string) as row_change
        from starts_join_ends
    )
select * from pairs_with_deltas
'''

df = get_results(cur, statement)
pickle.dump(df, open('../pickles/df_q1.pkl', 'wb'))
df.head()

,transaction_hash,nft_address,token_id,start_activity,start_address,start_value,start_timestamp,start_row,end_activity,end_address,end_value,end_timestamp,end_row,delta_value,delta_days,row_change
0,0x7c2300b8a34fc65605e045a7eef9ab4966c8de2e87bc...,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,0,mint,0xb776cAb26B9e6Be821842DC0cc0e8217489a4581,3.000000e+16,1630522006,1,hold,0xb776cAb26B9e6Be821842DC0cc0e8217489a4581,3.000000e+16,1632586540,1,0.0,23.895069,1->1
1,0x9fc3718924d43e27d6a3adc0716335cd72c70e72c4e1...,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,1,mint,0x8A73024B39A4477a5Dc43fD6360e446851AD1D28,1.500000e+17,1630523878,1,hold,0x8A73024B39A4477a5Dc43fD6360e446851AD1D28,1.500000e+17,1632586540,1,0.0,23.873403,1->1
2,0x2be3aae4638495c06b6b03800b6ebe165d11ba532a0e...,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,10,mint,0x5e5C817E9264B46cBBB980198684Ad9d14f3e0B4,1.500000e+17,1630524131,1,hold,0x5e5C817E9264B46cBBB980198684Ad9d14f3e0B4,1.500000e+17,1632586540,1,0.0,23.870475,1->1
3,0x30f16b319b1e5a7f35b370742684773af9b7fe3c5cbd...,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,11,mint,0x8376f63c13b99D3eedfA51ddd77Ff375279B3Ba0,3.000000e+16,1630524172,1,hold,0x8376f63c13b99D3eedfA51ddd77Ff375279B3Ba0,3.000000e+16,1632586540,1,0.0,23.870000,1->1
4,0xf8312cedb3f2404ea14e14396279fb55ca36399f9b77...,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,12,mint,0xb5e34552F32BA9226C987769BF6555a538510BA8,9.000000e+16,1630525084,1,hold,0xb5e34552F32BA9226C987769BF6555a538510BA8,9.000000e+16,1632586540,1,0.0,23.859444,1->1
